##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Treinamento distribuído com DTensors


<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/dtensor_ml_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/distribute/dtensor_ml_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/distribute/dtensor_ml_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tutorials/distribute/dtensor_ml_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Visão geral

O DTensor oferece uma maneira de distribuir o treinamento do seu modelo nos dispositivos para aumentar a eficiência, confiabilidade e escalabilidade. Confira mais detalhes sobre os conceitos do DTensor no [Guia de programação do DTensor](https://www.tensorflow.org/guide/dtensor_overview).

Neste tutorial, você treinará um modelo de Análise de Sentimentos com o DTensor. Três esquemas de treinamento distribuído são demonstrados neste exemplo:

- Treinamento Paralelo de Dados, em que as amostras do treinamento são fragmentadas (particionadas) nos dispositivos.
- Treinamento Paralelo de Modelo, em que as variáveis do modelo são fragmentadas nos dispositivos.
- Treinamento Paralelo Espacial, em que as características dos dados de entrada são fragmentadas nos dispositivos. (Também conhecido como [Particionamento Espacial](https://cloud.google.com/blog/products/ai-machine-learning/train-ml-models-on-large-images-and-3d-volumes-with-spatial-partitioning-on-cloud-tpus))

A parte de treinamento deste tutorial foi inspirada no notebook [Guia do Kaggle sobre Análise de Sentimentos](https://www.kaggle.com/code/anasofiauzsoy/yelp-review-sentiment-analysis-tensorflow-tfds/notebook). Para entender todo o fluxo de trabalho de treinamento e avaliação (sem o DTensor), confira esse notebook.

Veja abaixo as etapas deste tutorial:

- Primeiro comece com uma limpeza de dados para obter um `tf.data.Dataset` de frases divididas em tokens e sua polaridade.

- Em seguida, crie um modelo MLP com camadas Dense e BatchNorm personalizadas. Use um `tf.Module` para monitorar as variáveis de inferência. O construtor do modelo recebe argumentos `Layout` adicionais para controlar a fragmentação das variáveis.

- Para o treinamento, primeiro você usa o treinamento Paralelo de Dados junto com o recurso de checkpoint do `tf.experimental.dtensor`. Em seguida, você prossegue para o treinamento Paralelo de Modelo e o treinamento Paralelo Espacial.

- A seção final descreve brevemente a interação entre `tf.saved_model` e `tf.experimental.dtensor` (a partir do TensorFlow 2.9).


## Configuração

O DTensor faz parte da versão 2.9.0 do TensorFlow.

In [ ]:
!pip install --quiet --upgrade --pre tensorflow tensorflow-datasets

Em seguida, importe `tensorflow` e `tensorflow.experimental.dtensor`. Depois, configure o TensorFlow para usar 8 CPUs virtuais.

Embora este exemplo use CPUs, o DTensor funciona da mesma forma em dispositivos com CPU, GPU ou TPU.

In [ ]:
import tempfile
import numpy as np
import tensorflow_datasets as tfds

import tensorflow as tf

from tensorflow.experimental import dtensor
print('TensorFlow version:', tf.__version__)

In [ ]:
def configure_virtual_cpus(ncpu):
  phy_devices = tf.config.list_physical_devices('CPU')
  tf.config.set_logical_device_configuration(phy_devices[0], [
        tf.config.LogicalDeviceConfiguration(),
    ] * ncpu)

configure_virtual_cpus(8)
DEVICES = [f'CPU:{i}' for i in range(8)]

tf.config.list_logical_devices('CPU')

## Download do dataset

Baixe o conjunto de dados de avaliações do IMDB para treinar o modelo de Análise de Sentimentos.

In [ ]:
train_data = tfds.load('imdb_reviews', split='train', shuffle_files=True, batch_size=64)
train_data

## Preparação dos dados

Primeiro, divida o texto em tokens. Use uma extensão do one-hot encoding, o modo `“td_idf”` de `tf.keras.layers.TextVectorization`.

-     Por questões de velocidade, limite o número de tokens a 1.200.
- Para manter o `tf.Module` simples, execute `TextVectorization` em uma etapa de pré-processamento antes do treinamento.

O resultado final da seção de limpeza de dados é um `Dataset` com o texto dividido em tokens como `x` e o rótulo como `y`.

**Observação**: executar o `TextVectorization` em uma etapa de pré-processamento **não é uma prática comum nem recomendada**, pois, ao fazer isso, pressupõe-se que os dados de treinamento caibam na memória do cliente, o que nem sempre é o caso.


In [ ]:
text_vectorization = tf.keras.layers.TextVectorization(output_mode='tf_idf', max_tokens=1200, output_sequence_length=None)
text_vectorization.adapt(data=train_data.map(lambda x: x['text']))

In [ ]:
def vectorize(features):
  return text_vectorization(features['text']), features['label']

train_data_vec = train_data.map(vectorize)
train_data_vec

## Criação de uma rede neural com o DTensor

Agora, crie um uma rede Multi-Layer Perceptron (MLP) com o `DTensor`. A rede usará camadas Dense e BatchNorm totalmente conectadas.

O `DTensor` expande o TensorFlow por meio de uma expansão de SPMD (Single-Program Multi-Data – um programa, múltiplos dados) do TensorFlow Ops comum de acordo com os atributos `dtensor.Layout` do `Tensor` e das variáveis de entrada.

As variáveis das camadas com reconhecimento do `DTensor` são `dtensor.DVariable`, e os construtores de objetos de camadas com reconhecimento do `DTensor` recebem outras entradas `Layout` além dos parâmetros de camada usuais.

Observação: a partir do TensorFlow 2.9, as camadas do Keras, como `tf.keras.layer.Dense`, e `tf.keras.layer.BatchNormalization`, aceitam argumentos `dtensor.Layout`.  Consulte o [Tutorial de integração do Keras com o DTensor](/tutorials/distribute/dtensor_keras_tutorial) para ver mais informações sobre como usar o Keras com o DTensor.

### Camada Dense

A seguinte camada Dense personalizada define 2 variáveis de camada: $W_{ij}$ é a variável para os pesos, e $b_i$ é a variável para os bias.

$$ y_j = \sigma(\sum_i x_i W_{ij} + b_j) $$


### Dedução do Layout

Este resultado advém das seguintes observações:

- A fragmentação DTensor preferida para operandos do produto escalar de uma matriz $t_j = \sum_i x_i W_{ij}$ é a fragmentação de $\mathbf{W}$ e $\mathbf{x}$ da mesma forma no eixo $i$.

- A fragmentação DTensor preferida para operandos da soma de uma matriz $t_j + b_j$ é a fragmentação de $\mathbf{t}$ e $\mathbf{b}$ da mesma forma no eixo $j$.


In [ ]:
class Dense(tf.Module):

  def __init__(self, input_size, output_size,
               init_seed, weight_layout, activation=None):
    super().__init__()

    random_normal_initializer = tf.function(tf.random.stateless_normal)

    self.weight = dtensor.DVariable(
        dtensor.call_with_layout(
            random_normal_initializer, weight_layout,
            shape=[input_size, output_size],
            seed=init_seed
            ))
    if activation is None:
      activation = lambda x:x
    self.activation = activation
    
    # bias is sharded the same way as the last axis of weight.
    bias_layout = weight_layout.delete([0])

    self.bias = dtensor.DVariable(
        dtensor.call_with_layout(tf.zeros, bias_layout, [output_size]))

  def __call__(self, x):
    y = tf.matmul(x, self.weight) + self.bias
    y = self.activation(y)

    return y

### BatchNorm

Uma camada de normalização de lotes ajuda a evitar o colapso de modos durante o treinamento. Neste caso, a inclusão de camadas de normalização de lotes ajuda que o treinamento do modelo evite gerar um modelo que produza somente zeros.

O construtor da camada `BatchNorm` personalizada abaixo não recebe um argumento `Layout`. O motivo é que `BatchNorm` não tem variáveis de camada. Isso ainda funciona com o DTensor porque “x”, a única entrada da camada, já é um DTensor que representa o lote global.

Observação: com o DTensor, o Tensor de entrada “x” sempre representa o lote global. Portanto, `tf.nn.batch_normalization` é aplicado ao lote global. Isso é diferente do treinamento com `tf.distribute.MirroredStrategy`, em que o Tensor “x” representa somente o fragmento por réplica do lote (o lote local).

In [ ]:
class BatchNorm(tf.Module):

  def __init__(self):
    super().__init__()

  def __call__(self, x, training=True):
    if not training:
      # This branch is not used in the Tutorial.
      pass
    mean, variance = tf.nn.moments(x, axes=[0])
    return tf.nn.batch_normalization(x, mean, variance, 0.0, 1.0, 1e-5)

Uma camada de normalização de lotes completa (como `tf.keras.layers.BatchNormalization`) precisará de argumentos Layout para suas variáveis.

In [ ]:
def make_keras_bn(bn_layout):
  return tf.keras.layers.BatchNormalization(gamma_layout=bn_layout,
                                            beta_layout=bn_layout,
                                            moving_mean_layout=bn_layout,
                                            moving_variance_layout=bn_layout,
                                            fused=False)

### Junção das camadas

Em seguida, crie uma rede Multi-Layer Perceptron (MLP) com os blocos acima.  O diagrama abaixo mostra as relações dos eixos entre a entrada `x` e as matrizes de peso para as duas camadas `Dense` sem qualquer fragmentação ou replicação DTensor aplicada.

<img src="https://www.tensorflow.org/images/dtensor/no_dtensor.png" class="no-filter" alt="The input and weight matrices for a non distributed model.">


A saída da primeira camada `Dense` é passada para a entrada da segunda camada `Dense` (após o `BatchNorm`). Portanto, a fragmentação DTensor preferida para a saída da primeira camada `Dense` ($\mathbf{W_1}$) e para a entrada da segunda camada `Dense` ($\mathbf{W_2}$) é a fragmentação de $\mathbf{W_1}$ e $\mathbf{W_2}$ da mesma forma no eixo comum $\hat{j}$.

$$ \mathsf{Layout}[{W_{1,ij}}; i, j] = \left[\hat{i}, \hat{j}\right] \ \mathsf{Layout}[{W_{2,jk}}; j, k] = \left[\hat{j}, \hat{k} \right] $$

Embora a dedução do layout mostre que os 2 layouts não são independentes, para fins de simplicidade da interface do modelo, `MLP` receberá 2 argumentos `Layout`, um por camada Dense.

In [ ]:
from typing import Tuple

class MLP(tf.Module):

  def __init__(self, dense_layouts: Tuple[dtensor.Layout, dtensor.Layout]):
    super().__init__()

    self.dense1 = Dense(
        1200, 48, (1, 2), dense_layouts[0], activation=tf.nn.relu)
    self.bn = BatchNorm()
    self.dense2 = Dense(48, 2, (3, 4), dense_layouts[1])

  def __call__(self, x):
    y = x
    y = self.dense1(y)
    y = self.bn(y)
    y = self.dense2(y)
    return y


O equilíbrio entre a precisão das restrições de dedução de layout e a simplicidade da API é uma questão comum de design de APIs que usam o DTensor. Também é possível capturar a dependência entre os `Layouts` com uma API diferente. Por exemplo, a classe `MLPStricter` cria objetos `Layout` no construtor.

In [ ]:
class MLPStricter(tf.Module):

  def __init__(self, mesh, input_mesh_dim, inner_mesh_dim1, output_mesh_dim):
    super().__init__()

    self.dense1 = Dense(
        1200, 48, (1, 2), dtensor.Layout([input_mesh_dim, inner_mesh_dim1], mesh),
        activation=tf.nn.relu)
    self.bn = BatchNorm()
    self.dense2 = Dense(48, 2, (3, 4), dtensor.Layout([inner_mesh_dim1, output_mesh_dim], mesh))


  def __call__(self, x):
    y = x
    y = self.dense1(y)
    y = self.bn(y)
    y = self.dense2(y)
    return y

Para garantir que o modelo seja executado, coloque em seu modelo layouts completamente replicados e um lote completamente replicado de entrada `“x”`.

In [ ]:
WORLD = dtensor.create_mesh([("world", 8)], devices=DEVICES)

model = MLP([dtensor.Layout.replicated(WORLD, rank=2),
             dtensor.Layout.replicated(WORLD, rank=2)])

sample_x, sample_y = train_data_vec.take(1).get_single_element()
sample_x = dtensor.copy_to_mesh(sample_x, dtensor.Layout.replicated(WORLD, rank=2))
print(model(sample_x))

## Movimentação dos dados para o dispositivo

Geralmente, iteradores `tf.data` (e os outros métodos de busca de dados) produzem objetos tensor armazenados na memória local do dispositivo host. Esses dados precisam ser transferidos para a memória do dispositivo acelerador que armazena tensores do DTensor.

`dtensor.copy_to_mesh` não é apropriado para essa situação, pois replica tensores de entrada em todos os dispositivos devido à perspectiva global do DTensor. Portanto, neste tutorial, você usará uma função helper `repack_local_tensor` para permitir a transferência dos dados. A função helper usa o `dtensor.pack` para enviar (e somente enviar) o fragmento do lote global que é destinado a uma réplica para o dispositivo que armazena a réplica.

Esta função simplificada pressupõe um único cliente. Em uma aplicação multicliente, pode ser trabalhoso determinar a forma correta de dividir o tensor local e o mapeamento entre os segmentos da divisão e os dispositivos locais.

Está planejada uma API DTensor adicional para simplificar a integração com `tf.data` , com suporte a aplicações com um e vários clientes. Fique de olho.

In [ ]:
def repack_local_tensor(x, layout):
  """Repacks a local Tensor-like to a DTensor with layout.

  This function assumes a single-client application.
  """
  x = tf.convert_to_tensor(x)
  sharded_dims = []

  # For every sharded dimension, use tf.split to split the along the dimension.
  # The result is a nested list of split-tensors in queue[0].
  queue = [x]
  for axis, dim in enumerate(layout.sharding_specs):
    if dim == dtensor.UNSHARDED:
      continue
    num_splits = layout.shape[axis]
    queue = tf.nest.map_structure(lambda x: tf.split(x, num_splits, axis=axis), queue)
    sharded_dims.append(dim)

  # Now we can build the list of component tensors by looking up the location in
  # the nested list of split-tensors created in queue[0].
  components = []
  for locations in layout.mesh.local_device_locations():
    t = queue[0]
    for dim in sharded_dims:
      split_index = locations[dim]  # Only valid on single-client mesh.
      t = t[split_index]
    components.append(t)

  return dtensor.pack(components, layout)

## Treinamento Paralelo de Dados

Nesta seção, você treinará seu modelo MLP com dados do treinamento Paralelo de Dados. Seções posteriores demonstrarão os treinamentos Paralelo de Modelo e Paralelo Espacial.

O treinamento Paralelo de Dados é um esquema usado com frequência para aprendizado de máquina distribuído:

- As variáveis do modelo são replicadas em N dispositivos.
- Um lote global é dividido em N lotes por réplica.
- Cada lote por réplica é treinado no dispositivo da réplica.
- O gradiente é reduzido antes de os dados de peso serem aplicados coletivamente em todas as réplicas.

O treinamento Paralelo de Dados proporciona um speedup praticamente linear quanto ao número de dispositivos.

### Criação de uma malha para paralelismo de dados

Um loop de treinamento típico de Paralelo de Dados usa uma `Mesh` DTensor que consiste de uma única dimensão `batch`, em que cada dispositivo se torna uma réplica que recebe um fragmento do lote global.

<img src="https://www.tensorflow.org/images/dtensor/dtensor_data_para.png" class="no-filter" alt="Data parallel mesh">

O modelo replicado é executado na réplica e, portanto, as variáveis do modelo são completamente replicadas (não fragmentadas).

In [ ]:
mesh = dtensor.create_mesh([("batch", 8)], devices=DEVICES)

model = MLP([dtensor.Layout([dtensor.UNSHARDED, dtensor.UNSHARDED], mesh),
             dtensor.Layout([dtensor.UNSHARDED, dtensor.UNSHARDED], mesh),])


### Encapsulamento dos dados de treinamento em DTensors

O lote de dados de treinamento deve ser encapsulado em DTensors fragmentados no eixo de `“batch”`(primeiro) para que o DTensor distribua os dados de treinamento na dimensão de malha do `“batch”` de maneira uniforme.

**Observação:** no DTensor, o `batch size` sempre se refere ao tamanho global de lote. O tamanho do lote deve ser escolhido de uma forma que possa ser dividido pelo tamanho da dimensão de malha do `batch`, gerando um número inteiro.

In [ ]:
def repack_batch(x, y, mesh):
  x = repack_local_tensor(x, layout=dtensor.Layout(['batch', dtensor.UNSHARDED], mesh))
  y = repack_local_tensor(y, layout=dtensor.Layout(['batch'], mesh))
  return x, y

sample_x, sample_y = train_data_vec.take(1).get_single_element()
sample_x, sample_y = repack_batch(sample_x, sample_y, mesh)

print('x', sample_x[:, 0])
print('y', sample_y)

### Etapa de treinamento

Este exemplo usa um otimizador do Método do Gradiente Descendente Estocástico com um Loop de Treinamento Personalizado (CTL, na sigla em inglês). Confira o [guia de Loop de Treinamento Personalizado](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch) e o [Passo a passo](https://www.tensorflow.org/tutorials/customization/custom_training_walkthrough) para ver mais informações sobre esses tópicos.

O `train_step` é encapsulado como uma `tf.function` para indicar que esse corpo precisa ser traçado como um Grafo TensorFlow. O corpo de `train_step` consiste de um passo de inferência para frente, um passo de gradiente para trás e a atualização das variáveis.

O corpo de `train_step` não contém nenhuma anotação DTensor especial. Em vez disso, `train_step` contém somente operações TensorFlow de alto nível que processam a entrada `x` e `y` da visão global do lote de entrada e o modelo. Todas as anotações do DTensor (`Mesh`, `Layout`) são contabilizadas fora do passo de treinamento.

In [ ]:
# Refer to the CTL (custom training loop guide)
@tf.function
def train_step(model, x, y, learning_rate=tf.constant(1e-4)):
  with tf.GradientTape() as tape:
    logits = model(x)
    # tf.reduce_sum sums the batch sharded per-example loss to a replicated
    # global loss (scalar).
    loss = tf.reduce_sum(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=y))
  parameters = model.trainable_variables
  gradients = tape.gradient(loss, parameters)
  for parameter, parameter_gradient in zip(parameters, gradients):
    parameter.assign_sub(learning_rate * parameter_gradient)

  # Define some metrics
  accuracy = 1.0 - tf.reduce_sum(tf.cast(tf.argmax(logits, axis=-1, output_type=tf.int64) != y, tf.float32)) / x.shape[0]
  loss_per_sample = loss / len(x)
  return {'loss': loss_per_sample, 'accuracy': accuracy}

### Criação de checkpoints

Você pode criar um checkpoint de um modelo DTensor usando `tf.train.Checkpoint`. Ao salvar e restaurar DVariables fragmentadas, será feito um processo fragmentado eficiente de salvamento e restauração. Atualmente, ao usar `tf.train.Checkpoint.save` e `tf.train.Checkpoint.restore`, todas as DVariables devem estar na mesma malha de host, e DVariables e variáveis não podem ser salvas juntas. Saiba mais sobre a criação de checkpoints [neste guia](../../guide/checkpoint.ipynb).

Quando um checkpoint DTensor é restaurado, os `Layout`s de variáveis podem ser diferentes em relação a quando o checkpoint foi salvo. Ou seja, salvar modelos DTensor é independente de malhas e layouts, afetando somente a eficiência do salvamento fragmentado. Você pode salvar um modelo DTensor com uma determinada malha e layout e restaurá-lo em uma malha e layout diferentes. Este tutorial usa esse recurso para continuar o treinamento nas seções Treinamento Paralelo de Modelo e Treinamento Paralelo Espacial.


In [ ]:
CHECKPOINT_DIR = tempfile.mkdtemp()

def start_checkpoint_manager(model):
  ckpt = tf.train.Checkpoint(root=model)
  manager = tf.train.CheckpointManager(ckpt, CHECKPOINT_DIR, max_to_keep=3)

  if manager.latest_checkpoint:
    print("Restoring a checkpoint")
    ckpt.restore(manager.latest_checkpoint).assert_consumed()
  else:
    print("New training")
  return manager


### Loop de treinamento

Para o esquema do treinamento Paralelo de Dados, utilize épocas e gere relatórios do progresso. Três épocas não são suficientes para treinar o modelo – uma precisão de 50% é tão boa quanto tentar adivinhar aleatoriamente.

Ative os checkpoints para que você possa continuar o treinamento posteriormente. Na próxima seção, você carregará um checkpoint e fará o treinamento com um esquema paralelo diferente.

In [ ]:
num_epochs = 2
manager = start_checkpoint_manager(model)

for epoch in range(num_epochs):
  step = 0
  pbar = tf.keras.utils.Progbar(target=int(train_data_vec.cardinality()), stateful_metrics=[])
  metrics = {'epoch': epoch}
  for x,y in train_data_vec:

    x, y = repack_batch(x, y, mesh)

    metrics.update(train_step(model, x, y, 1e-2))

    pbar.update(step, values=metrics.items(), finalize=False)
    step += 1
  manager.save()
  pbar.update(step, values=metrics.items(), finalize=True)

## Treinamento Paralelo de Modelo

Se você mudar para uma `Mesh` bidimensional e fragmentar as variáveis do modelo na segunda dimensão da malha, então o treinamento vira um Paralelo de Modelo.

No treinamento Paralelo de Modelo, cada réplica do modelo fica em diversos dispositivos (2, neste caso):

- Há 4 réplicas do modelo, e o lote de dados de treinamento é distribuído para as 4 réplicas.
- Os 2 dispositivos dentro de uma única réplica do modelo recebem os dados de treinamento replicados.

<img src="https://www.tensorflow.org/images/dtensor/dtensor_model_para.png" class="no-filter" alt="Model parallel mesh">


In [ ]:
mesh = dtensor.create_mesh([("batch", 4), ("model", 2)], devices=DEVICES)
model = MLP([dtensor.Layout([dtensor.UNSHARDED, "model"], mesh), 
             dtensor.Layout(["model", dtensor.UNSHARDED], mesh)])

Como os dados de treinamento ainda são fragmentados na dimensão do lote, você pode reutilizar a mesma função `repack_batch` usada no caso do treinamento Paralelo de Dados. O DTensor vai replicar automaticamente o lote por réplica em todos os dispositivos dentro da réplica na dimensão de malha `"model"`.

In [ ]:
def repack_batch(x, y, mesh):
  x = repack_local_tensor(x, layout=dtensor.Layout(['batch', dtensor.UNSHARDED], mesh))
  y = repack_local_tensor(y, layout=dtensor.Layout(['batch'], mesh))
  return x, y

Em seguida, execute o loop de treinamento. O loop de treinamento reutiliza o mesmo gerenciador de checkpoints usado no exemplo do treinamento Paralelo de Dados, e o código é idêntico.

Você pode continuar treinando o modelo Paralelo de Dados treinado no treinamento Paralelo de Modelo.

In [ ]:
num_epochs = 2
manager = start_checkpoint_manager(model)

for epoch in range(num_epochs):
  step = 0
  pbar = tf.keras.utils.Progbar(target=int(train_data_vec.cardinality()))
  metrics = {'epoch': epoch}
  for x,y in train_data_vec:
    x, y = repack_batch(x, y, mesh)
    metrics.update(train_step(model, x, y, 1e-2))
    pbar.update(step, values=metrics.items(), finalize=False)
    step += 1
  manager.save()
  pbar.update(step, values=metrics.items(), finalize=True)

## Treinamento Paralelo Espacial

Ao treinar dados com uma dimensionalidade muito alta (por exemplo, uma imagem ou vídeo muito grande), pode ser interessante fragmentar na dimensão de característica. Isso se chama [Particionamento Espacial](https://cloud.google.com/blog/products/ai-machine-learning/train-ml-models-on-large-images-and-3d-volumes-with-spatial-partitioning-on-cloud-tpus), lançado inicialmente no TensorFlow para modelos de treinamento com grandes amostras de entrada 3D.

<img src="https://www.tensorflow.org/images/dtensor/dtensor_spatial_para.png" class="no-filter" alt="Spatial parallel mesh">

O DTensor também é compatível com esse caso de uso. A única mudança necessária é a criação de uma Malha que inclua uma dimensão de `feature` e a aplicação do `Layout` correspondente.


In [ ]:
mesh = dtensor.create_mesh([("batch", 2), ("feature", 2), ("model", 2)], devices=DEVICES)
model = MLP([dtensor.Layout(["feature", "model"], mesh), 
             dtensor.Layout(["model", dtensor.UNSHARDED], mesh)])


Fragmente os dados de entrada na dimensão de `feature` ao encapsular os tensores de entrada no DTensors. É possível fazer isso com uma função de reencapsulamento ligeiramente diferente, `repack_batch_for_spt`, em que `spt` significa Treinamento Paralelo Espacial.

In [ ]:
def repack_batch_for_spt(x, y, mesh):
    # Shard data on feature dimension, too
    x = repack_local_tensor(x, layout=dtensor.Layout(["batch", 'feature'], mesh))
    y = repack_local_tensor(y, layout=dtensor.Layout(["batch"], mesh))
    return x, y

O Treinamento Paralelo Espacial também pode continuar de um checkpoint criado com outros esquemas de treinamento paralelo.

In [ ]:
num_epochs = 2

manager = start_checkpoint_manager(model)
for epoch in range(num_epochs):
  step = 0
  metrics = {'epoch': epoch}
  pbar = tf.keras.utils.Progbar(target=int(train_data_vec.cardinality()))

  for x, y in train_data_vec:
    x, y = repack_batch_for_spt(x, y, mesh)
    metrics.update(train_step(model, x, y, 1e-2))

    pbar.update(step, values=metrics.items(), finalize=False)
    step += 1
  manager.save()
  pbar.update(step, values=metrics.items(), finalize=True)

## SavedModel e DTensor

A integração entre o DTensor e o SavedModel ainda está em desenvolvimento.

A partir do TensorFlow `2.11`, `tf.saved_model` pode salvar modelos DTensor fragmentados e replicados e, ao salvar, será feito um salvamento fragmentado eficiente em diferentes dispositivos da malha. Entretanto, após um modelo ser salvo, todas as anotações DTensor são perdidas, e as assinaturas salvas só podem ser usadas com Tensors comuns, não DTensors.

In [ ]:
mesh = dtensor.create_mesh([("world", 1)], devices=DEVICES[:1])
mlp = MLP([dtensor.Layout([dtensor.UNSHARDED, dtensor.UNSHARDED], mesh), 
           dtensor.Layout([dtensor.UNSHARDED, dtensor.UNSHARDED], mesh)])

manager = start_checkpoint_manager(mlp)

model_for_saving = tf.keras.Sequential([
  text_vectorization,
  mlp
])

@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def run(inputs):
  return {'result': model_for_saving(inputs)}

tf.saved_model.save(
    model_for_saving, "/tmp/saved_model",
    signatures=run)

A partir do TensorFlow 2.9.0, você pode fazer uma chamada a uma assinatura carregada somente com um Tensor comum ou um DTensor completamente replicado (que será convertido em um Tensor comum).

In [ ]:
sample_batch = train_data.take(1).get_single_element()
sample_batch

In [ ]:
loaded = tf.saved_model.load("/tmp/saved_model")

run_sig = loaded.signatures["serving_default"]
result = run_sig(sample_batch['text'])['result']

In [ ]:
np.mean(tf.argmax(result, axis=-1) == sample_batch['label'])

## Quais são os próximos passos?

Este tutorial demonstrou a criação e o treinamento de um modelo MLP de Análise de Sentimentos com o DTensor.

Por meio de primitivos `Mesh` e `Layout`, o DTensor pode transformar uma `tf.function` do TensorFlow em um programa distribuído para diversos esquemas de treinamento.

Em uma aplicação de aprendizado de máquina do mundo real, a avaliação e a validação cruzada devem ser usadas para evitar a geração de um modelo sobreajustado. As técnicas apresentadas neste tutorial também podem ser aplicadas para acrescentar paralelismo à avaliação.

É muito trabalhoso criar um modelo com `tf.Module` do zero, e reutilizar blocos existentes, como camadas e funções helper, pode acelerar drasticamente o desenvolvimento do modelo. A partir do TensorFlow 2.9, todas as camadas do Keras em `tf.keras.layers` aceitam Layouts do DTensor como argumentos e podem ser usados para criar modelos do DTensor. Você pode até mesmo reutilizar diretamente um modelo do Keras com o DTensor sem modificar a implementação do modelo. Consulte o [Tutorial de integração do Keras com o DTensor](https://www.tensorflow.org/tutorials/distribute/dtensor_keras_tutorial) para ver mais informações sobre como usar o Keras com o DTensor. 